In [1]:
import numpy as np 
import pandas as pd 
import mlflow
import mlflow.xgboost
from mlflow.models import infer_signature
from mlflow.tracking import MlflowClient
from sklearn.model_selection import TimeSeriesSplit
from sktime.performance_metrics.forecasting import (MeanAbsoluteScaledError, 
    MeanAbsolutePercentageError, MeanAbsoluteError, MeanSquaredError)
from xgboost import XGBRegressor

import optuna
from optuna.integration.mlflow import MLflowCallback
import joblib

from typing import Optional, Dict, Tuple, Literal
from enefit_challenge.models.forecaster import Forecaster

import warnings
warnings.filterwarnings('ignore')


TRACKING_URI = "http://127.0.0.1:5000/" # local tracking URI -> launch mlflow before training 


/Users/dylantartarini/miniconda3/envs/enefit/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In this notebook we're going to do two things:
1. inform the XGBoostForecaster of the uselessness of some features, according to previous model's iteration feature importance 
2. define a new subset of parameters to run experiments on, according to mlflow's parallel coordinates chart


In [2]:
from enefit_challenge.utils.dataset import load_enefit_training_data
not_feature_columns = ['datetime', 'row_id','prediction_unit_id','date','time', 'data_block_id']
cat_columns = ['county', 'product_type']
to_drop_cols = [
    '10_metre_u_wind_component_mean_f',
    '10_metre_v_wind_component_min_f',
    'cloudcover_low_mean_f',
    'dayofweek_sine',
    'direct_solar_radiation_max_f',
    'eic_count',
    'euros_per_mwh', # not sure about this one
    'hour_sine',
    'temperature_min_f',
    'total_precipitation_max_f',
    'week_sine',
    '10_metre_u_wind_component_min_f',
    '10_metre_v_wind_component_max_f',
    '10_metre_v_wind_component_std_f',
    'cloudcover_high_mean_f',
    'cloudcover_high_std_f',
    'cloudcover_low_min_f',
    'cloudcover_low_std_f',
    'cloudcover_mid_std_f',
    'cloudcover_total_std_f',
    'county_12', # find a way to drop
    'county_3', # find a way to drop
    'county_9', # find a way to drop
    'direct_solar_radiation_min_f',
    'direct_solar_radiation_std_f',
    'highest_price_per_mwh', # not sure about this one
    # 'installed_capacity', #this one should alpo be dropped?
    'month_cosine',
    'product_type_3', # find a way to drop
    'snowfall_max_f',
    'temperature_std_f',
    'total_precipitation_min_f',
    'total_precipitation_std_f',
    # 'year'
]
df_train = load_enefit_training_data()

In [5]:
class XGBoostForecaster(Forecaster):
    """
        Implementation of a Forecaster using `XGBRegressor` as base model, 
        `optuna` for hyperparameters optimization and `mlflow` as backend to track experiments
        and register best-in-class model for time series prediction.
    """
    def __init__(self)-> None:
        self.tracking_uri = mlflow.set_tracking_uri(TRACKING_URI)
        pass

    def fit_model(
        self,  
        X:pd.DataFrame,
        y:pd.Series,
        params:Optional[Dict]=None,
    ) -> XGBRegressor:
        """
        Trains a `XGBRegressor`

        -------     
        params:
        -------
        `X`:`pd.DataFrame`
            Features to use for fitting
        `y`:`pd.Series`
            Target variable
        `params`: `Optional[Dict]`
            optional dictionary of parameters to use
        -------     
        returns:
        -------
        fitted `XGBRegressor`
        """
        model = XGBRegressor(
            n_estimators=100, 
            objective='reg:squarederror',
            eval_metric='mae'
        )
        if params:
            model.set_params(**params)

        model.fit(X, y)
    
        return model
    
    def fit_and_test_fold(
        self, 
        params:Dict,
        X: pd.DataFrame, 
        y: pd.Series, 
        year_month_train, 
        year_month_test,
        experiment_name: str="xgboost",
        artifact_path: str="xgboost_model",
        metrics: list=["mae"]
    ) -> float:
        """
        Used for cross validation on different time splits; 
        also in charge of logging every experiment run / study trial into the backend.
        """
        
        first_dates_month = pd.to_datetime(X[['year', 'month']].assign(day=1))
        train_index = first_dates_month.isin(year_month_train)
        test_index = first_dates_month.isin(year_month_test)

        X_train = X[train_index]
        X_test = X[test_index]
        y_train = y[train_index]
        y_test = y[test_index]

        # fit model on training data
        mlflow.xgboost.autolog(log_datasets=False, log_models=False)
        model = self.fit_model(
            X_train, 
            y_train, 
            params
        )
        
        # generate predictions
        y_test_pred = model.predict(X_test)
        self.signature = infer_signature(X_train, y_test_pred)
        MAE = MeanAbsoluteError()
        mae = MAE(y_test, y_test_pred)
        MASE = MeanAbsoluteScaledError()
        mase = MASE(y_test, y_test_pred, y_train=y_train)
        MAPE = MeanAbsolutePercentageError()
        mape = MAPE(y_test, y_test_pred)
        MSE = MeanSquaredError()
        mse = MSE(y_test, y_test_pred)
        RMSE = MeanSquaredError(square_root=True)
        rmse = RMSE(y_test, y_test_pred)

        mlflow.xgboost.log_model(
            model, 
            artifact_path=artifact_path,
            signature=self.signature
        )
        mlflow.log_params(params)

        return mae, mase, mse, rmse, mape

    def train_model(
        self, 
        train_df: pd.DataFrame, 
        target_col: str,
        model_name: str,
        exclude_cols: list=[],
        categorical_features: list=[],
        experiment_name: str="xgboost",
        artifact_path: str="xgboost_model",
        params: Optional[Dict]=None
    ) -> None:
        """ 
        Takes an instance of `XGBRegressor` model and tracks the hyperparameter tuning
        experiment on training set using `mlflow` and `optuna`.  
        Registers the best version of the model according to a specified metric (to be implemented).
        
        -------     
        params:
        -------
        `experiment_name`: `str`
            the name of the experiment used to store runs in mlflow, 
            as well as the name of the optuna study
        `model_name`: `str`
            the name the final model will have in the registry
        `train_df`: `pd.DataFrame`
            the training data for the model.
        `target_col`: `str`
            the time-series target column
        `exclude_cols`: `list`  
            columns in dataset that should not be used
        `categorical_features`: `list`
            list of categorical features in the dataset
        `artifact_path`: `str`
            the path pointing to the mlflow artifact
        `params`: `Optional[Dict]`
            optional dictionary of parameters to use
        """
        self.model_name = model_name

        if len(categorical_features) > 0: 
           train_df = pd.get_dummies(train_df, columns=categorical_features)

        X = train_df.drop([target_col] + exclude_cols, axis=1)
        y = train_df[target_col]
        # unique year-month combinations -> to be used in cross-validation
        timesteps = np.sort(np.array(
            pd.to_datetime(X[['year', 'month']].assign(day=1)).unique().tolist()
        ))
        
        # define mlflow callback Handler for optuna 
        mlflc = MLflowCallback(
            metric_name=["MAE"]
        )
    
        @mlflc.track_in_mlflow() # decorator to allow mlflow logging
        def objective(trial):
            params = {
                'eval_metric': 'mae',
                'n_estimators': trial.suggest_int('n_estimators', 50, 200, log=True),
                'eta': trial.suggest_float('eta', 0.01, 0.2,log=True),
                'num_leaves': trial.suggest_int('num_leaves', 20, 50, log=True),
                'max_depth': trial.suggest_int('max_depth', 1, 15, log=True),
                # 'min_child_weight': trial.suggest_int('min_child_weight', 1, 25, log=True),
                'min_child_weight': trial.suggest_float('min_child_weight', 0.005, 0.05, log=True),
                'colsample_bytree': trial.suggest_float("colsample_bytree", 0.5, 1, log=True),
                'colsample_bylevel': trial.suggest_float("colsample_bylevel", 0.5, 1, log=True),
                'colsample_bynode': trial.suggest_float("colsample_bynode", 0.5, 1, log=True),
                'subsample': trial.suggest_float("subsample", 0.5, 0.85, log=True),
                'lambda': trial.suggest_float('lambda', 1, 5, log=True),
                'alpha': trial.suggest_float('alpha', 1e-3, 0.3, log=True),
                'num_boost_round': trial.suggest_int('num_boost_round', 10, 100, log=True)

            }
            cv = TimeSeriesSplit(n_splits=3) # cross validation
            cv_mae = [None]*3
            cv_mase = [None]*3
            cv_mse = [None]*3
            cv_rmse = [None]*3
            cv_mape = [None]*3
            for i, (train_index, test_index) in enumerate(cv.split(timesteps)):
                cv_mae[i], cv_mase[i], cv_mse[i], cv_rmse[i], cv_mape[i] = self.fit_and_test_fold(
                    params,
                    X, 
                    y, 
                    timesteps[train_index], 
                    timesteps[test_index]
                )
            trial.set_user_attr('split_mae', cv_mae)
            trial.set_user_attr('split_mase', cv_mase)
            trial.set_user_attr('split_mse', cv_mse)
            trial.set_user_attr('split_rmse', cv_rmse)
            trial.set_user_attr('split_mape', cv_mape)

            mlflow.log_metrics(
                {
                    "MAE":np.mean(cv_mae),
                    "MASE": np.mean(cv_mase),
                    "MSE": np.mean(cv_mse),
                    "RMSE":np.mean(cv_rmse),
                    "MAPE":np.mean(cv_mape)
                }
            )
            return np.mean(cv_mae) 

        
        sampler = optuna.samplers.TPESampler(
            n_startup_trials=10, 
            seed=0
        )

        self.study = optuna.create_study(
            directions=['minimize'],
            sampler=sampler,
            study_name=experiment_name
        )

        self.study.optimize(objective, n_trials=100, timeout= 7200, callbacks=[mlflc]) 
        
        # # search for the best run at the end of the experiment # not implemented now bc of callback bug
        # best_run = mlflow.search_runs(max_results=1,order_by=["metrics.MAE"]).run_id
        # # register new model version in mlflow
        # self.result = mlflow.register_model(
        #     model_uri=f"runs:/{best_run}/{artifact_path}",
        #     name=self.model_name
        # )

    def forecast(
        self, 
        input_data: pd.DataFrame,
        use_best_from_run: bool=True,
        use_env_model: Literal["Staging", "Production", None]=None,
        use_version: int=None
        ) -> pd.DataFrame:
        """ 
        Fetches a version of the model from the mlflow backend and uses it
        to perform prediction on new input data.  
        What version is used depends on params settings, 
        defaults to using the best version from the last experiment run (currently not implemented). 
        -------     
        params:
        -------
        `input_data`: `pd.DataFrame`
            the input data for prediction,
              must have the same schema as what's in the model's signature.
        `use_best_from_run`: `bool=True`      
            use the best model from the current series of iterations, defaults to True
        `use_env_model`: `Literal["Staging", "Production", None]=None`
            use model from a given mlflow environment, defaults to None.  
            Said model might come from past iterations, depending on what you decide in the UI
        `use_version`: `int=None`
            use a previously trained version of the model. 
            Said version must have been registered from a previous iteration,  
            either by the UI or with mlflow's API
        """
        if use_best_from_run:
            # not implemented now bc of callback bug
            use_prod_model=None
            use_version=None
        
            # model = mlflow.pyfunc.load_model(
            #     model_uri=f"models:/{self.model_name}/{self.result.version}"
            # )
            # y_pred = model.predict(input_data)
            # return y_pred
        
        if use_env_model is not None:
            use_version = None

            model = mlflow.pyfunc.load_model(
                # get registered model in given environment
                model_uri=f"models:/{self.model_name}/{use_env_model}"
            )
            y_pred = model.predict(input_data)
            return y_pred

        if use_version is not None:
            # get specific registered version of model
            model = mlflow.pyfunc.load_model(
                model_uri=f"models:/{self.model_name}/{use_version}"
            )
            y_pred = model.predict(input_data)
            return y_pred

        
        if (not use_best_from_run) & (use_env_model is None) & (use_version is None):
            return ValueError(
                    "You must specify which kind of XGBoostForecaster you intend to use for prediction"
            )
        

In [6]:
xgb = XGBoostForecaster()

xgb.train_model(
    train_df=df_train,
    target_col="target",
    model_name="xgboost_enefit",
    exclude_cols=not_feature_columns+to_drop_cols,
    categorical_features=cat_columns
)

[I 2023-12-09 20:23:38,209] A new study created in memory with name: xgboost


[20:23:40] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1697107917112/work/src/learner.cc:767: 
Parameters: { "num_boost_round", "num_leaves" } are not used.



2023/12/09 20:23:51 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during xgboost autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 19e7208b57a54e678c17bf7f738629cf. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Duplicate parameter keys have been submitted: [\'num_boost_round\']. Please ensure the request contains only one param value per param key.")]')]


[20:23:59] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1697107917112/work/src/learner.cc:767: 
Parameters: { "num_boost_round", "num_leaves" } are not used.



2023/12/09 20:24:23 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during xgboost autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 19e7208b57a54e678c17bf7f738629cf. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Duplicate parameter keys have been submitted: [\'num_boost_round\']. Please ensure the request contains only one param value per param key.")]')]


...

2023/12/09 22:22:29 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during xgboost autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 690a0fdf8e504d3dbf4929b307fa19b7. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Duplicate parameter keys have been submitted: [\'num_boost_round\']. Please ensure the request contains only one param value per param key.")]')]


[22:22:38] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1697107917112/work/src/learner.cc:767: 
Parameters: { "num_boost_round", "num_leaves" } are not used.



2023/12/09 22:26:52 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during xgboost autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 690a0fdf8e504d3dbf4929b307fa19b7. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Duplicate parameter keys have been submitted: [\'num_boost_round\']. Please ensure the request contains only one param value per param key.")]')]


[22:27:04] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1697107917112/work/src/learner.cc:767: 
Parameters: { "num_boost_round", "num_leaves" } are not used.



2023/12/09 22:33:30 WARNING mlflow.utils.autologging_utils: Encountered unexpected error during xgboost autologging: The following failures occurred while performing one or more logging operations: [MlflowException('Failed to perform one or more operations on the run with ID 690a0fdf8e504d3dbf4929b307fa19b7. Failed operations: [RestException("INVALID_PARAMETER_VALUE: Duplicate parameter keys have been submitted: [\'num_boost_round\']. Please ensure the request contains only one param value per param key.")]')]
[I 2023-12-09 22:33:34,981] Trial 33 finished with value: 59.29531452049374 and parameters: {'n_estimators': 129, 'eta': 0.03714140496061684, 'num_leaves': 33, 'max_depth': 15, 'min_child_weight': 0.03799711813097755, 'colsample_bytree': 0.9833836345820283, 'colsample_bylevel': 0.7202423724379589, 'colsample_bynode': 0.7269053652706522, 'subsample': 0.8349316272575732, 'lambda': 1.1418152438001463, 'alpha': 0.017584632896750593, 'num_boost_round': 27}. Best is trial 31 with value